In [1]:
%matplotlib inline

# 构建神经网络

神经网络由对数据执行操作的层/模块组成。
[torch.nn](https://pytorch.org/docs/stable/nn.html) 命名空间提供了您需要的所有构建块
建立自己的神经网络。 PyTorch 中的每个模块都是 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) 的子类。
神经网络是一个模块本身，由其他模块（层）组成。 这种嵌套结构允许
轻松构建和管理复杂的架构。

在以下部分中，我们将构建一个神经网络来对 FashionMNIST 数据集中的图像进行分类。

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get a hardware device for training

We want to be able to train our model on a hardware accelerator like the GPU, 
if it is available. Let's check to see if 
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we 
continue to use the CPU.

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## 定义类

我们通过子类化 `nn.Module` 来定义我们的神经网络，并且
在 `__init__` 中初始化神经网络层。 每个 `nn.Module` 子类都实现
`forward` 方法中对输入数据的操作。

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建一个“神经网络”的实例，并将其移动到“设备”，并打印它的结构。



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


为了使用模型，我们将输入数据传递给它。 这将执行模型的 `forward`，
以及一些[后台操作]（https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866）。
不要直接调用`model.forward()`！

在输入上调用模型会返回一个 10 维张量，其中包含每个类的原始预测值。
我们通过将其传递给 `nn.Softmax` 模块的一个实例来获得预测密度。

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) 
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5])


## 模型层

我们初始化 [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
层将每个 28x28 的 2D 图像转换为 784 个像素值的连续数组（保持小批量维度（dim=0））。



In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

[线性层](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
是一个模块，它使用存储的权重和偏差对输入应用线性变换。



In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear 

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using it's stored weights and biases.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

非线性激活是在模型的输入和输出之间创建复杂映射的原因。
它们在线性变换后应用以引入*非线性*，帮助神经网络
学习各种各样的现象。

在这个模型中，我们在我们之间使用 [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
线性层，但还有其他激活可以在您的模型中引入非线性。

In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2620,  0.5372, -0.3142,  0.5595, -0.0338, -0.2578, -0.1453,  0.0142,
         -0.3144, -0.1978,  0.1119, -0.4031, -0.5338, -0.2112,  0.0843, -0.1039,
         -0.6157, -0.3176,  0.2991,  0.3278],
        [ 0.0082,  0.2950, -0.3428,  0.2677,  0.1682, -0.1624,  0.0630,  0.2772,
         -0.3598, -0.1056,  0.3218, -0.0650, -0.7976, -0.1417,  0.1265, -0.2078,
         -0.6100, -0.1949,  0.5933,  0.6934],
        [ 0.0833,  0.5182, -0.4482,  0.1808, -0.1582, -0.3140, -0.1534,  0.1163,
         -0.0906, -0.2653,  0.4350,  0.0649, -0.8912, -0.3647,  0.2512, -0.1568,
         -0.2083, -0.1839,  0.2909, -0.1341]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.2620, 0.5372, 0.0000, 0.5595, 0.0000, 0.0000, 0.0000, 0.0142, 0.0000,
         0.0000, 0.1119, 0.0000, 0.0000, 0.0000, 0.0843, 0.0000, 0.0000, 0.0000,
         0.2991, 0.3278],
        [0.0082, 0.2950, 0.0000, 0.2677, 0.1682, 0.0000, 0.0630, 0.2772, 0.0000,
         0.0000, 0.3218, 0.0000, 0.0000, 0.0000, 0.126

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) 是有序的
模块的容器。 数据按照定义的相同顺序通过所有模块。 您可以使用
顺序容器将快速网络组合在一起，如“seq_modules”。



In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

神经网络的最后一个线性层返回 `logits` - \[`-infty`, `infty`] 中的原始值 - 这些值被传递给
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) 模块。 logits 被缩放到值 \[0, 1\] 代表模型对每个类别的预测密度。 `dim` 参数指示值必须总和为 1 的维度。

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## 神经网络内的许多层都是*参数化的*，即具有相关的权重
和在训练期间优化的偏差。 自动子类化`nn.Module`
跟踪模型对象中定义的所有字段，并使所有参数
可以使用模型的`parameters()` 或`named_parameters()` 方法访问。

在这个例子中，我们迭代每个参数，并打印它的大小和它的值的预览。


In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0318,  0.0107,  0.0324,  ...,  0.0006, -0.0194, -0.0082],
        [ 0.0058, -0.0337,  0.0020,  ...,  0.0117, -0.0323,  0.0208]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0105,  0.0231], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0344,  0.0338,  0.0201,  ..., -0.0415,  0.0139,  0.0285],
        [ 0.0307, -0.0436, -0.0363,  ..., -0.0211, -0.0438, -0.0257]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | Size: to

--------------


